# Bed Frame Analysis with Home Assistant Events

This notebook uses the refactored Python modules for analyzing bed frame sensor data with Home Assistant events.

In [ ]:
# Import required libraries and our custom modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# Import our refactored modules
from movement_detection import MovementConfig
from ha_client import load_ha_events
from bed_frame_analysis import (
    load_sensor_data,
    analyze_all_events,
    print_analysis_summary
)
from visualization import plot_pitch_with_events, plot_velocity_profile

%matplotlib inline

In [ ]:
# Load and examine sensor data
df = load_sensor_data('open-close.csv')

print(f"Loaded {len(df)} sensor readings")
print(f"Sensor time range: {df['sensor_timestamp'].min()} to {df['sensor_timestamp'].max()}")
print(f"Pitch range: {df['pitch'].min():.3f} to {df['pitch'].max():.3f} radians")

# Display first few rows
print("\nFirst 5 rows of data:")
display(df[['sensor_timestamp', 'pitch', 'roll', 'yaw']].head())

In [ ]:
# Load Home Assistant events
ENTITY_ID = 'cover.bed_remote_head_position'
HA_URL = 'http://tvpi:8123'
TOKEN_PATH = '/run/secrets/home-assistant/token'

ha_events = load_ha_events(
    sensor_df=df,
    entity_id=ENTITY_ID,
    ha_url=HA_URL,
    token_path=TOKEN_PATH
)

print(f"Found {len(ha_events)} events within sensor data timeframe")
for event in ha_events:
    print(f"  {event['timestamp']}: {event['state']}")

In [ ]:
# Configure and run movement analysis
config = MovementConfig(
    window_seconds=20.0,          # Look for movement within 20 seconds
    movement_threshold=0.005,      # Radians/second to consider as "stopped"
    stability_time=0.5,            # Seconds of stability required to confirm stop
    start_movement_threshold=0.01  # Minimum change to detect movement has started
)

# Analyze all events
analysis = analyze_all_events(ha_events, df, config)

# Print detailed results
print_analysis_summary(analysis)

In [ ]:
# Create main visualization
fig = plot_pitch_with_events(df, ha_events, analysis, figsize=(14, 8))
plt.show()

# Print summary of detected stop points
if analysis['results']:
    print(f"\nDetected {len(analysis['results'])} movement stop points")
    avg_start_delay = np.mean([r['start_delay_ms'] for r in analysis['results']])
    avg_stop_delay = np.mean([r['stop_delay_ms'] for r in analysis['results']])
    print(f"Average start delay: {avg_start_delay:.0f} ms")
    print(f"Average stop delay: {avg_stop_delay:.0f} ms")

In [ ]:
# Plot velocity profiles for individual events
for i, event in enumerate(ha_events[:2]):  # Show first 2 events
    print(f"\nEvent {i+1}: {event['state'].capitalize()} at {event['timestamp'].strftime('%H:%M:%S')}")
    fig = plot_velocity_profile(event, df, config.window_seconds, figsize=(12, 6))
    plt.show()

In [ ]:
# Interactive exploration - Try different configurations
print("Testing different movement detection parameters:\n")

configs_to_test = [
    ("Sensitive", MovementConfig(movement_threshold=0.003, stability_time=0.3)),
    ("Default", MovementConfig()),
    ("Less Sensitive", MovementConfig(movement_threshold=0.008, stability_time=0.7)),
]

comparison_results = []

for name, test_config in configs_to_test:
    test_analysis = analyze_all_events(ha_events, df, test_config)
    
    print(f"{name} Configuration:")
    print(f"  Movement threshold: {test_config.movement_threshold:.3f} rad/s")
    print(f"  Stability time: {test_config.stability_time:.1f} s")
    
    if test_analysis['results']:
        avg_stop = np.mean([r['stop_delay_ms'] for r in test_analysis['results']])
        print(f"  Detected events: {len(test_analysis['results'])}")
        print(f"  Avg stop delay: {avg_stop:.0f} ms")
    else:
        print(f"  No events detected")
    
    comparison_results.append((name, test_config, test_analysis))
    print()

In [ ]:
# Export results for further analysis
if analysis['results']:
    # Convert results to DataFrame for easy manipulation
    results_df = pd.DataFrame(analysis['results'])
    
    # Add derived columns
    results_df['movement_duration_ms'] = results_df['stop_delay_ms'] - results_df['start_delay_ms']
    results_df['pitch_change_abs'] = results_df['pitch_change'].abs()
    
    print("Analysis Results Summary:")
    print(results_df[['event_state', 'start_delay_ms', 'stop_delay_ms', 
                      'movement_duration_ms', 'pitch_change', 'direction']])
    
    # Save to CSV if needed
    # results_df.to_csv('movement_analysis_results.csv', index=False)
    # print("\nResults saved to 'movement_analysis_results.csv'")